In [1]:
import networkx as nx
import numpy as np
import pandas as pd
# ------------------- Data Loading -------------------
# Load the station data
stations_file_path = 'London_stations.csv'  # Path to the CSV file
stations_data = pd.read_csv(stations_file_path)

# Ensure column names are stripped of extra spaces
stations_data.columns = stations_data.columns.str.strip()

# Create a dictionary to map station names to their (latitude, longitude) pairs
stations = dict(zip(stations_data['Station'], zip(stations_data['Latitude'], stations_data['Longitude'])))

# ------------------- Haversine Function -------------------
# Haversine formula to calculate the distance between two latitude/longitude points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])  # Convert to radians
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # Return distance in kilometers

# ------------------- Create Tube Graph -------------------
def create_tube_graph():
    # Initialize the graph
    G = nx.Graph()

    # Add nodes to the graph (stations with coordinates)
    G.add_nodes_from(stations)

    # New connections (stations pairs)
    connections = [
        ("Waterloo", "Embankment"),
        ("Embankment", "Charing Cross"),
        ("Charing Cross", "Piccadilly Circus"),
        ("Piccadilly Circus", "Oxford Circus"),
        ("Oxford Circus", "Regent's Park"),
        ("Regent's Park", "Baker Street"),
        
        ("Marble Arch", "Bond Street"),
        ("Bond Street", "Oxford Circus"),
        ("Oxford Circus", "Tottenham Court Road"),
        ("Tottenham Court Road", "Holborn"),
        ("Holborn", "Chancery Lane"),

        ("Baker Street", "Bond Street"),
        ("Bond Street", "Green Park"),
        ("Green Park", "Westminster"),
        ("Westminster", "Waterloo"),
        ("Waterloo", "Southwark"),

        ("Hyde Park Corner", "Green Park"),
        ("Green Park", "Piccadilly Circus"),
        ("Piccadilly Circus", "Leicester Square"),
        ("Leicester Square", "Covent Garden"),
        ("Covent Garden", "Holborn"),
        ("Holborn", "Russell Square")
    ]

    # Add edges with distances calculated using the Haversine formula
    for station1, station2 in connections:
        if station1 in stations and station2 in stations:
            lat1, lon1 = stations[station1]
            lat2, lon2 = stations[station2]
            distance = haversine(lat1, lon1, lat2, lon2)
            G.add_edge(station1, station2, weight=distance)

    return G

# Dijkstra's algorithm to find the shortest path based on calculated distances
def dijkstra_shortest_path(G, source, target):
    # Use Dijkstra's algorithm to find the shortest path from source to target
    length, path = nx.single_source_dijkstra(G, source, target)
    return length, path

# ------------------- Main -------------------
if __name__ == "__main__":
    # Create the London Tube graph using data from CSV
    G = create_tube_graph()

    # Set source and target stations
    source_station = "Marble Arch"
    target_station = "Charing Cross"

    # Find the shortest path and its length
    distance, path = dijkstra_shortest_path(G, source_station, target_station)

    # Output the result
    print(f"Shortest path from {source_station} to {target_station}:")
    print(f"Path: {path}")
    print(f"Total distance: {distance:.2f} kilometers")

Shortest path from Marble Arch to Charing Cross:
Path: ['Marble Arch', 'Bond Street', 'Oxford Circus', 'Piccadilly Circus', 'Charing Cross']
Total distance: 2.53 kilometers
